In [1]:
from pathlib import Path
import importlib

import numpy as np
import pandas as pd
from astropy.io import ascii
import matplotlib.pyplot as plt

from dask.distributed import Client
import dask.array
from dask.dataframe.utils import make_meta

from hats import read_hats
from hats.inspection import plot_pixels
from hats_import.catalog.file_readers import CsvReader
from hats_import.margin_cache.margin_cache_arguments import MarginCacheArguments
from hats_import.pipeline import ImportArguments, pipeline_with_client

import lsdb

from catalog_filtering import bandFilterLenient, contains_PM
import hpms_pipeline as hpms

print("Imported libraries.")

Imported libraries.


In [27]:
bandList = ['G','R','I','Z','Y']
class_star = None
spread_model = 0.05
magnitude_error = 0.05
check_flags = True
mag = 19
query_string = bandFilterLenient(bandList,classStar=class_star,spreadModel=spread_model,magError=magnitude_error,flag=check_flags,mag=mag)
des_cols = (
    [f'CLASS_STAR_{band}' for band in bandList] + 
    [f'FLAGS_{band}' for band in bandList] + 
    ['RA','DEC','COADD_OBJECT_ID'] + 
    [f'SPREAD_MODEL_{band}' for band in bandList] + 
    [f'WAVG_MAG_PSF_{band}' for band in bandList] + 
    [f'WAVG_MAGERR_PSF_{band}' for band in bandList]
)
k = 1
max_obj_deviation = 1000000
pm_speed_min = 2000 #units are milliseconds per year
pm_speed_max = 10**5
des_id_col = 'COADD_OBJECT_ID_1'
mag_cols = [f'WAVG_MAG_PSF_{band}' for band in ['G','I']]
cone_search_rad = 50
min_neighbors = 3
max_neighbor_dist = 18
milliarc_degree_conversion = 1/(1000*3600)
xmatch_max_neighbors = 100
print("Defined local vars.")

Defined local vars.


In [3]:
# Change to the directories where the data will be stored
CATALOG_DIR = Path("../../../../catalogs")
MARGIN_CACHE_DIR = CATALOG_DIR / 'margin_caches'

GAIA_NAME = "gaia_dr3_pm_greater_100"
GAIA_DIR = CATALOG_DIR / GAIA_NAME

DES_NAME = "des_light"
DES_DIR = CATALOG_DIR / DES_NAME 

DES_X_GAIA_NAME = "des_dr2_x_gaia_dr3"
DES_X_GAIA_DIR = CATALOG_DIR / DES_X_GAIA_NAME
DES_MARGIN_CACHE_NAME = "des_margin_cache_18_arcsec"
DES_MARGIN_CACHE_DIR = MARGIN_CACHE_DIR / DES_MARGIN_CACHE_NAME

print("Defined directories.")

Defined directories.


In [4]:
des_catalog = lsdb.read_hats(DES_DIR)
gaia_catalog = lsdb.read_hats(GAIA_DIR)
xmatch_catalog = lsdb.read_hats(DES_X_GAIA_DIR, columns='all')
print("Defined catalogs.")

Defined catalogs.


In [5]:
%%time
pm_filter_xmatch = xmatch_catalog.query(f'{pm_speed_max**2} >(pmra_gaia**2 + pmdec_gaia**2) > {pm_speed_min**2}')
with Client():
    df = pm_filter_xmatch.compute()

df

/ocean/projects/phy210048p/jpassos/conda-venvs/lsdb-main/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33629 instead
  warnings.warn(
2025-06-24 15:18:45,266 - distributed.worker - ERROR - Failed to communicate with scheduler during heartbeat.
Traceback (most recent call last):
  File "/ocean/projects/phy210048p/jpassos/conda-venvs/lsdb-main/lib/python3.12/site-packages/distributed/comm/tcp.py", line 226, in read
    frames_nosplit_nbytes_bin = await stream.read_bytes(fmt_size)
                                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/ocean/projects/phy210048p/jpassos/conda-venvs/lsdb-main/lib/python3.12/site-packages/distributed/worker.py", line 1269, in heartbeat
    response = await ret

CPU times: user 32.8 s, sys: 961 ms, total: 33.8 s
Wall time: 1min 21s


,CLASS_STAR_G_des,CLASS_STAR_R_des,CLASS_STAR_I_des,CLASS_STAR_Z_des,CLASS_STAR_Y_des,FLAGS_G_des,FLAGS_R_des,FLAGS_I_des,FLAGS_Z_des,FLAGS_Y_des,RA_des,DEC_des,COADD_OBJECT_ID_des,SPREAD_MODEL_G_des,SPREAD_MODEL_R_des,SPREAD_MODEL_I_des,SPREAD_MODEL_Z_des,SPREAD_MODEL_Y_des,WAVG_MAG_PSF_G_des,WAVG_MAG_PSF_R_des,WAVG_MAG_PSF_I_des,WAVG_MAG_PSF_Z_des,WAVG_MAG_PSF_Y_des,WAVG_MAGERR_PSF_G_des,WAVG_MAGERR_PSF_R_des,WAVG_MAGERR_PSF_I_des,WAVG_MAGERR_PSF_Z_des,WAVG_MAGERR_PSF_Y_des,NEPOCHS_G_des,NEPOCHS_R_des,NEPOCHS_I_des,NEPOCHS_Z_des,NEPOCHS_Y_des,source_id_gaia,ra_gaia,dec_gaia,pmra_gaia,pmdec_gaia,phot_g_mean_mag_gaia,phot_bp_mean_mag_gaia,phot_rp_mean_mag_gaia,_dist_arcsec
_healpix_29,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1153482605725265461,0.844888,0.845371,0.844807,0.845333,0.844827,3,3,3,3,3,1.386363,-37.369781,1043295027,-0.015229,0.040008,0.037639,0.034238,0.050002,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,0,0,0,0,0,2306965202564744064,1.383284,-37.367744,5633.438088,-2334.721273,7.682494,8.802319,6.61627,11.461677
1153482609302817738,0.84542,0.845421,0.845413,0.845344,0.035287,19,19,19,19,19,1.382708,-37.367553,1043296795,0.025377,0.024557,0.017375,0.016744,0.025608,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,0,0,0,0,0,2306965202564744064,1.383284,-37.367744,5633.438088,-2334.721273,7.682494,8.802319,6.61627,1.786142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3206298013938319236,0.001864,0.861837,0.986711,0.059186,0.994785,2,2,2,2,2,331.071881,-56.792344,949223282,-0.017833,0.00133,0.002163,0.036238,0.008708,23.579453,21.088121,17.886377,-99.0,13.975197,0.115939,0.012154,0.001204,-99.0,0.000657,1,3,4,0,2,6412596012146801152,331.076453,-56.793812,3981.976663,-2466.831815,18.030664,21.142073,16.060108,10.448522
3206298016903817602,0.768001,0.13328,0.076799,0.102573,0.260082,1,1,1,1,1,331.073253,-56.790655,949223665,0.010093,0.018254,0.007055,-0.004826,-0.005615,-99.0,23.490891,23.837128,-99.0,-99.0,-99.0,0.13595,0.236819,-99.0,-99.0,0,2,1,0,0,6412596012146801152,331.076453,-56.793812,3981.976663,-2466.831815,18.030664,21.142073,16.060108,12.998833


In [6]:
df_no_dupes = df[~df['source_id_gaia'].duplicated(keep='first')]

gaia_ids = df['source_id_gaia']

#dropping because otherwise produces error when performing .apply below
df_no_dupes = df_no_dupes.drop('source_id_gaia', axis=1)

df_no_dupes

,CLASS_STAR_G_des,CLASS_STAR_R_des,CLASS_STAR_I_des,CLASS_STAR_Z_des,CLASS_STAR_Y_des,FLAGS_G_des,FLAGS_R_des,FLAGS_I_des,FLAGS_Z_des,FLAGS_Y_des,RA_des,DEC_des,COADD_OBJECT_ID_des,SPREAD_MODEL_G_des,SPREAD_MODEL_R_des,SPREAD_MODEL_I_des,SPREAD_MODEL_Z_des,SPREAD_MODEL_Y_des,WAVG_MAG_PSF_G_des,WAVG_MAG_PSF_R_des,WAVG_MAG_PSF_I_des,WAVG_MAG_PSF_Z_des,WAVG_MAG_PSF_Y_des,WAVG_MAGERR_PSF_G_des,WAVG_MAGERR_PSF_R_des,WAVG_MAGERR_PSF_I_des,WAVG_MAGERR_PSF_Z_des,WAVG_MAGERR_PSF_Y_des,NEPOCHS_G_des,NEPOCHS_R_des,NEPOCHS_I_des,NEPOCHS_Z_des,NEPOCHS_Y_des,ra_gaia,dec_gaia,pmra_gaia,pmdec_gaia,phot_g_mean_mag_gaia,phot_bp_mean_mag_gaia,phot_rp_mean_mag_gaia,_dist_arcsec
_healpix_29,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1153482605725265461,0.844888,0.845371,0.844807,0.845333,0.844827,3,3,3,3,3,1.386363,-37.369781,1043295027,-0.015229,0.040008,0.037639,0.034238,0.050002,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,0,0,0,0,0,1.383284,-37.367744,5633.438088,-2334.721273,7.682494,8.802319,6.61627,11.461677
1257518643390044839,0.844564,0.845075,0.845251,0.843732,0.801461,2,2,2,2,2,33.079341,3.567016,1250008459,0.028251,0.024656,0.022653,0.022333,0.032334,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,0,0,0,0,0,33.079599,3.567385,-1762.405718,-1852.8711,9.205841,10.280927,8.165198,1.62095
2368327529620177120,0.3251,0.177878,0.293946,0.140421,0.001102,0,0,0,0,0,53.568008,-49.893871,1398391104,-0.006986,0.000464,0.004162,0.024926,0.02906,24.553566,23.715429,23.094166,22.574404,-99.0,0.236298,0.062562,0.070822,0.11552,-99.0,1,9,6,2,0,53.567196,-49.890084,2360.592206,483.127504,15.837138,19.476444,14.289389,13.763072
2390050329526096144,0.125803,0.120065,0.028601,0.028627,0.028627,3,3,3,3,3,62.610625,-53.614224,1464092812,0.010486,0.038736,0.037149,0.040352,0.041889,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,0,0,0,0,0,62.611,-53.612997,-825.17937,-2415.577565,11.902864,13.809194,10.605541,4.490123
2405297220004419732,0.194207,0.84529,0.84489,0.845284,0.029913,3,3,3,3,3,77.964238,-45.04643,1540617639,0.008839,0.006246,0.002662,0.001698,0.005954,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,0,0,0,0,0,77.959937,-45.043813,6491.223339,-5708.61415,8.063552,9.086244,7.059598,14.437522
2423978658324676571,0.845425,0.845425,0.845406,0.845371,0.84513,22,22,18,18,22,50.000054,-43.066641,1445898840,-0.000132,-0.000009,0.000011,0.000009,0.000002,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,0,0,0,0,0,50.000344,-43.066553,3035.017316,726.964482,4.063915,4.440891,3.508293,0.825611
2450054482295873745,0.844839,0.845014,0.845117,0.845267,0.844982,2,2,2,2,6,5.037242,-64.869108,1054999950,0.000474,0.000276,-0.000397,0.000404,1.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,0,0,0,0,0,5.03561,-64.869617,1706.746855,1164.959443,4.07351,4.369477,3.600201,3.096527
2468500457058623057,0.84536,0.029197,0.845341,0.842975,0.047746,2,3,2,2,2,32.623122,-50.820946,1243465193,0.00557,0.00185,0.001703,0.001036,0.004741,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,0,0,0,0,0,32.622946,-50.820906,2125.416147,637.975043,5.893562,6.311407,5.297973,0.425698
2468500457058623057,0.84536,0.029197,0.845341,0.842975,0.047746,2,3,2,2,2,32.623122,-50.820946,1243465193,0.00557,0.00185,0.001703,0.001036,0.004741,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,0,0,0,0,0,32.624069,-50.820823,2168.886011,710.847727,13.08884,<NA>,<NA>,2.199757


In [42]:
def algo_found_pm(row, des_cols, cone_search_rad, 
                  query_string, max_neighbor_dist, k,
                  max_obj_deviation, id_col, xmatch_max_neighbors,
                  mag_cols, min_neighbors):    
    # Filter DES around PM of interest:
    catalog = (
        lsdb.read_hats(DES_DIR, columns=des_cols, margin_cache=DES_MARGIN_CACHE_DIR)
        .cone_search(ra=row['ra_gaia'], dec=row['dec_gaia'], radius_arcsec=cone_search_rad)
    )
    
    with Client():
        filtered_df = hpms.execute_pipeline(catalog, query_string, xmatch_max_neighbors, 
                                   max_neighbor_dist, min_neighbors, k, max_obj_deviation, id_col, mag_cols).compute()
    if not filtered_df.empty:
        min_idx = filtered_df['kth_min_deviation'].idxmin()
        best_row = filtered_df.loc[min_idx]
        return pd.Series({
            'kth_min_deviation': best_row['kth_min_deviation'].min(),
            'max_obj_distance': best_row['max_obj_distance'].max(),
            'max_mag_diff': best_row['max_mag_diff'].max()
        })
    else:
        print('EMPTY')
        return pd.Series({
            'kth_min_deviation': np.nan,
            'max_obj_distance': np.nan,
            'max_mag_diff': np.nan
        })

In [43]:
importlib.reload(hpms)
print("Done")

Done


In [44]:
res = df_no_dupes.apply(func=algo_found_pm, axis=1, des_cols=des_cols, cone_search_rad=cone_search_rad, 
                        query_string=query_string, max_neighbor_dist=max_neighbor_dist, k=k, 
                        max_obj_deviation=max_obj_deviation, id_col=des_id_col, mag_cols=mag_cols, min_neighbors=min_neighbors,
                        xmatch_max_neighbors=xmatch_max_neighbors)
res

/ocean/projects/phy210048p/jpassos/conda-venvs/lsdb-main/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46055 instead
  warnings.warn(


['CLASS_STAR_G_2', 'CLASS_STAR_R_2', 'CLASS_STAR_I_2', 'CLASS_STAR_Z_2', 'CLASS_STAR_Y_2', 'FLAGS_G_2', 'FLAGS_R_2', 'FLAGS_I_2', 'FLAGS_Z_2', 'FLAGS_Y_2', 'RA_2', 'DEC_2', 'COADD_OBJECT_ID_2', 'SPREAD_MODEL_G_2', 'SPREAD_MODEL_R_2', 'SPREAD_MODEL_I_2', 'SPREAD_MODEL_Z_2', 'SPREAD_MODEL_Y_2', 'WAVG_MAG_PSF_G_2', 'WAVG_MAG_PSF_R_2', 'WAVG_MAG_PSF_I_2', 'WAVG_MAG_PSF_Z_2', 'WAVG_MAG_PSF_Y_2', 'WAVG_MAGERR_PSF_G_2', 'WAVG_MAGERR_PSF_R_2', 'WAVG_MAGERR_PSF_I_2', 'WAVG_MAGERR_PSF_Z_2', 'WAVG_MAGERR_PSF_Y_2', 'kth_min_deviation', 'max_obj_distance', 'max_mag_diff'] next Index(['CLASS_STAR_G_1', 'CLASS_STAR_R_1', 'CLASS_STAR_I_1', 'CLASS_STAR_Z_1',
       'CLASS_STAR_Y_1', 'FLAGS_G_1', 'FLAGS_R_1', 'FLAGS_I_1', 'FLAGS_Z_1',
       'FLAGS_Y_1', 'RA_1', 'DEC_1', 'COADD_OBJECT_ID_1', 'SPREAD_MODEL_G_1',
       'SPREAD_MODEL_R_1', 'SPREAD_MODEL_I_1', 'SPREAD_MODEL_Z_1',
       'SPREAD_MODEL_Y_1', 'WAVG_MAG_PSF_G_1', 'WAVG_MAG_PSF_R_1',
       'WAVG_MAG_PSF_I_1', 'WAVG_MAG_PSF_Z_1', 'WAVG_MAG_PSF_

/ocean/projects/phy210048p/jpassos/conda-venvs/lsdb-main/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41789 instead
  warnings.warn(


['CLASS_STAR_G_2', 'CLASS_STAR_R_2', 'CLASS_STAR_I_2', 'CLASS_STAR_Z_2', 'CLASS_STAR_Y_2', 'FLAGS_G_2', 'FLAGS_R_2', 'FLAGS_I_2', 'FLAGS_Z_2', 'FLAGS_Y_2', 'RA_2', 'DEC_2', 'COADD_OBJECT_ID_2', 'SPREAD_MODEL_G_2', 'SPREAD_MODEL_R_2', 'SPREAD_MODEL_I_2', 'SPREAD_MODEL_Z_2', 'SPREAD_MODEL_Y_2', 'WAVG_MAG_PSF_G_2', 'WAVG_MAG_PSF_R_2', 'WAVG_MAG_PSF_I_2', 'WAVG_MAG_PSF_Z_2', 'WAVG_MAG_PSF_Y_2', 'WAVG_MAGERR_PSF_G_2', 'WAVG_MAGERR_PSF_R_2', 'WAVG_MAGERR_PSF_I_2', 'WAVG_MAGERR_PSF_Z_2', 'WAVG_MAGERR_PSF_Y_2', 'kth_min_deviation', 'max_obj_distance', 'max_mag_diff'] next Index(['CLASS_STAR_G_1', 'CLASS_STAR_R_1', 'CLASS_STAR_I_1', 'CLASS_STAR_Z_1',
       'CLASS_STAR_Y_1', 'FLAGS_G_1', 'FLAGS_R_1', 'FLAGS_I_1', 'FLAGS_Z_1',
       'FLAGS_Y_1', 'RA_1', 'DEC_1', 'COADD_OBJECT_ID_1', 'SPREAD_MODEL_G_1',
       'SPREAD_MODEL_R_1', 'SPREAD_MODEL_I_1', 'SPREAD_MODEL_Z_1',
       'SPREAD_MODEL_Y_1', 'WAVG_MAG_PSF_G_1', 'WAVG_MAG_PSF_R_1',
       'WAVG_MAG_PSF_I_1', 'WAVG_MAG_PSF_Z_1', 'WAVG_MAG_PSF_

/ocean/projects/phy210048p/jpassos/conda-venvs/lsdb-main/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39653 instead
  warnings.warn(


['CLASS_STAR_G_2', 'CLASS_STAR_R_2', 'CLASS_STAR_I_2', 'CLASS_STAR_Z_2', 'CLASS_STAR_Y_2', 'FLAGS_G_2', 'FLAGS_R_2', 'FLAGS_I_2', 'FLAGS_Z_2', 'FLAGS_Y_2', 'RA_2', 'DEC_2', 'COADD_OBJECT_ID_2', 'SPREAD_MODEL_G_2', 'SPREAD_MODEL_R_2', 'SPREAD_MODEL_I_2', 'SPREAD_MODEL_Z_2', 'SPREAD_MODEL_Y_2', 'WAVG_MAG_PSF_G_2', 'WAVG_MAG_PSF_R_2', 'WAVG_MAG_PSF_I_2', 'WAVG_MAG_PSF_Z_2', 'WAVG_MAG_PSF_Y_2', 'WAVG_MAGERR_PSF_G_2', 'WAVG_MAGERR_PSF_R_2', 'WAVG_MAGERR_PSF_I_2', 'WAVG_MAGERR_PSF_Z_2', 'WAVG_MAGERR_PSF_Y_2', 'kth_min_deviation', 'max_obj_distance', 'max_mag_diff'] next Index(['CLASS_STAR_G_1', 'CLASS_STAR_R_1', 'CLASS_STAR_I_1', 'CLASS_STAR_Z_1',
       'CLASS_STAR_Y_1', 'FLAGS_G_1', 'FLAGS_R_1', 'FLAGS_I_1', 'FLAGS_Z_1',
       'FLAGS_Y_1', 'RA_1', 'DEC_1', 'COADD_OBJECT_ID_1', 'SPREAD_MODEL_G_1',
       'SPREAD_MODEL_R_1', 'SPREAD_MODEL_I_1', 'SPREAD_MODEL_Z_1',
       'SPREAD_MODEL_Y_1', 'WAVG_MAG_PSF_G_1', 'WAVG_MAG_PSF_R_1',
       'WAVG_MAG_PSF_I_1', 'WAVG_MAG_PSF_Z_1', 'WAVG_MAG_PSF_

2025-06-24 15:32:33,422 - distributed.worker - ERROR - Compute Failed
Key:       ('nestedframe-fused-lambda-0110fca65e6c521a8038a4ccab04369b', 1)
State:     executing
Task:  <Task ('nestedframe-fused-lambda-0110fca65e6c521a8038a4ccab04369b', 1) _execute_subgraph(...)>
Exception: 'AssertionError()'
Traceback: '  File "/ocean/projects/phy210048p/jpassos/conda-venvs/lsdb-main/lib/python3.12/site-packages/dask/dataframe/core.py", line 98, in apply_and_enforce\n    df = func(*args, **kwargs)\n         ^^^^^^^^^^^^^^^^^^^^^\n  File "/ocean/projects/phy210048p/jpassos/astrophysics/Jupyter Notebooks/kth_star_pipeline/prototyping/hpms_pipeline.py", line 265, in apply_kth_star\n    for col in hpms_col_names: assert(col in df.columns)\n                                      ^^^^^^^^^^^^^^^^^\n'

2025-06-24 15:32:33,467 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('nestedframe-fused-lambda-0110fca65e6c521a8038a4ccab04369b', 0))" coro=<Worker.ex

['CLASS_STAR_G_2', 'CLASS_STAR_R_2', 'CLASS_STAR_I_2', 'CLASS_STAR_Z_2', 'CLASS_STAR_Y_2', 'FLAGS_G_2', 'FLAGS_R_2', 'FLAGS_I_2', 'FLAGS_Z_2', 'FLAGS_Y_2', 'RA_2', 'DEC_2', 'COADD_OBJECT_ID_2', 'SPREAD_MODEL_G_2', 'SPREAD_MODEL_R_2', 'SPREAD_MODEL_I_2', 'SPREAD_MODEL_Z_2', 'SPREAD_MODEL_Y_2', 'WAVG_MAG_PSF_G_2', 'WAVG_MAG_PSF_R_2', 'WAVG_MAG_PSF_I_2', 'WAVG_MAG_PSF_Z_2', 'WAVG_MAG_PSF_Y_2', 'WAVG_MAGERR_PSF_G_2', 'WAVG_MAGERR_PSF_R_2', 'WAVG_MAGERR_PSF_I_2', 'WAVG_MAGERR_PSF_Z_2', 'WAVG_MAGERR_PSF_Y_2', 'kth_min_deviation', 'max_obj_distance', 'max_mag_diff'] next Index(['_healpix_29', 'CLASS_STAR_G_1', 'CLASS_STAR_R_1', 'CLASS_STAR_I_1',
       'CLASS_STAR_Z_1', 'CLASS_STAR_Y_1', 'FLAGS_G_1', 'FLAGS_R_1',
       'FLAGS_I_1', 'FLAGS_Z_1', 'FLAGS_Y_1', 'RA_1', 'DEC_1',
       'COADD_OBJECT_ID_1', 'SPREAD_MODEL_G_1', 'SPREAD_MODEL_R_1',
       'SPREAD_MODEL_I_1', 'SPREAD_MODEL_Z_1', 'SPREAD_MODEL_Y_1',
       'WAVG_MAG_PSF_G_1', 'WAVG_MAG_PSF_R_1', 'WAVG_MAG_PSF_I_1',
       'WAVG_MAG_PS

AssertionError: 